# Exporting image volumes from Harmony

This notebook is designed to take the raw export from the Harmony software and tile the individual images together into a mosaic, both across time and the z-axis, and export that as a multichannel .TIFF image for movie creation in FIJI or _Napari_. 



In [1]:
import napari
import matplotlib.pyplot as plt
import glob
import os
import octopusheavy as octo
import pandas as pd
from tqdm.auto import tqdm
from functools import partial
from octopusheavy import tile
from skimage.io import imread, imsave
import dask.array as da

In [1]:
import os, glob
import octopuslite as octo

### Find images

The images will be in the folder labelled `Images`. Please input the path to these images below.

In [2]:
image_dir = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/'
fns = glob.glob(os.path.join(image_dir, '*.tiff'))
print(len(fns), 'image files found')

138402 image files found


### Loading metadata

The metadata file should match the filepattern `Index.idx.xml`. Please input the path to this metadata file below.

In [3]:
metadata_fn = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Index.idx.xml'
df = octo.utils.read_harmony_metadata(metadata_fn)

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Done!


### Assay layout metadata (optional)

If you want to know which plate wells correspond to which experiment conditions, input the path to the Assay Layout metadata here. It will be an xml file in a folder named `AssayLayout`.

In [6]:
assay_metadata_fn = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml'
assay_layout = octo.utils.read_harmony_metadata(assay_metadata_fn, assay_layout=True)
assay_layout

Reading metadata XML file...
Done!


Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

In [9]:
len(assay_layout)

24

In [8]:
row_col_list = list()
for index, row in (df.iterrows()):
    row_col_list.append(tuple((int(row['Row']), int(row['Col']))))
row_col_list = list(set(row_col_list))
for n, i in enumerate(row_col_list):
    print('Position index and (row,column):', n, i)

Position index and (row,column): 0 (3, 4)
Position index and (row,column): 1 (4, 3)
Position index and (row,column): 2 (4, 9)
Position index and (row,column): 3 (3, 7)
Position index and (row,column): 4 (5, 4)
Position index and (row,column): 5 (4, 6)
Position index and (row,column): 6 (3, 10)
Position index and (row,column): 7 (5, 7)
Position index and (row,column): 8 (6, 5)
Position index and (row,column): 9 (6, 8)
Position index and (row,column): 10 (4, 5)
Position index and (row,column): 11 (3, 3)
Position index and (row,column): 12 (3, 9)
Position index and (row,column): 13 (5, 6)
Position index and (row,column): 14 (4, 8)
Position index and (row,column): 15 (3, 6)
Position index and (row,column): 16 (5, 9)
Position index and (row,column): 17 (6, 4)
Position index and (row,column): 18 (6, 7)
Position index and (row,column): 19 (4, 7)
Position index and (row,column): 20 (3, 5)
Position index and (row,column): 21 (4, 4)
Position index and (row,column): 22 (4, 10)
Position index and 

## Get dimensionality of image volume

In [ ]:
channel_IDs = df['ChannelID'].unique()
plane_IDs = df['PlaneID'].unique()
timepoint_IDs = df['TimepointID'].unique()

## Set mosaic parameters

The `chunk_fraction` is how many sections you want one slice cut up into (has to be a square) and the `_load_image` partial function can include any image transformations you wish (border crop or background removal etc).

In [ ]:
chunk_fraction = 49
load_transform_image = partial(tile.load_image, transforms=[])#input_transforms)

## Set segmentation parameters

In [ ]:
!nvcc --version
!nvidia-smi

from cellpose import core, utils, io, models, metrics

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

model = models.Cellpose(gpu=True, model_type='cyto')

In [ ]:
def segment(img):
    masks, flows, styles, diams = model.eval(img, diameter=200, channels=[0,0],
                                             flow_threshold=None, cellprob_threshold=0)
    return masks

# Compile and segment at the same time...? 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
lazy_hack = {1:(slice(4032, 6048), slice(2016, 4032)),
            2:(slice(0, 2016), slice(0, 2016)),
            3:(slice(0, 2016), slice(2016, 4032)),
            4:(slice(0, 2016), slice(4032, 6048)),
            5:(slice(2016, 4032), slice(4032, 6048)),
            6:(slice(2016, 4032), slice(2016, 4032)),
            7:(slice(2016, 4032), slice(0, 2016)),
            8:(slice(4032, 6048), slice(0, 2016)),
            9:(slice(4032, 6048), slice(4032, 6048)),
            }
import re
regex = re.compile('f([0-9]*)')
from skimage.io import imsave

In [ ]:
for row, col in tqdm(reversed(row_col_list), desc = 'Position progress'):
    print(row, col)

In [ ]:
for row, col in tqdm(reversed(row_col_list), desc = 'Position progress'):
    print(row, col)

# running in reverse

In [ ]:
col

In [ ]:
z_mask_stack = []
t_mask_stack = []
# images = dict()
# masks = dict()
# for row, col in tqdm(row_col_list, desc = 'Position progress'):
# for row, col in tqdm(reversed(row_col_list), desc = 'Position progress'):
    ### define row and col
#     row, col = str(row), str(col)
row, col = '3', '8'
### clear empty arrays for organsing into dask arrays
t_stack = []
t_mask_stack = []
### iterate over each individual image slice, tiling together
for time in tqdm(timepoint_IDs, leave = False, desc = 'Timepoint progress'):
    c_stack = []
    for channel in tqdm(channel_IDs, leave = False, desc = 'Channel progress'):
        z_stack = []
        if channel == '1':
            z_mask_stack = []
        for plane in tqdm(plane_IDs, leave = False, desc = 'Z-slice progress'):
            frame, chunk_info = tile.stitch(load_transform_image,
                                df, 
                                image_dir, 
                                time, 
                                plane, 
                                channel, 
                                row, 
                                col, 
                                chunk_fraction)
            ### if we're iterating over the GFP channel then segment that
            if channel == '1':



                masks_frame = segment(frame)
                ### convert to chunked dask array
                masks_frame = da.from_array(masks_frame, chunks=frame.chunksize)
                ### append to z stack
                z_mask_stack.append(masks_frame)
                ### use chunk information to iterative save out individual tiles
                for chunk in chunk_info:
                    fn = chunk.fuse_info['file']
#                         x, y = chunk.fuse_info['transform'][0:2,2].astype(int)
#                         crops = tuple((slice(x, x+2016), slice(y, y+2016)))
                    ID = int((regex.findall(fn))[0])
                    crops = lazy_hack[ID]
                    save_out_mask = masks_frame[crops]
                    new_fn = fn.replace('ch1', 'ch99')
                    if new_fn == fn:
                        raise Exception(f"Tried overwriting file {fn}") 
                    else:
                        imsave(new_fn, save_out_mask, check_contrast=False)
            ### collect stitched frames together into time stack
            z_stack.append(frame)
        ### stack channel series together (images)
        c_stack.append(z_stack)
    ### stack together timewise
    t_stack.append(c_stack)
    t_mask_stack.append(z_mask_stack)
### stack stitched dask arrays together into multidim image volumes
images[(int(row), int(col))] = da.stack([da.stack(c_stack, axis = 0) for c_stack in t_stack])
masks[(int(row), int(col))] = da.stack(t_mask_stack)

In [ ]:
new_fn

In [ ]:
frame, chunk_info = tile.stitch(load_transform_image,
                                    df, 
                                    image_dir, 
                                    time, 
                                    plane, 
                                    channel, 
                                    row, 
                                    col, 
                                    chunk_fraction)

In [ ]:
from skimage.io import imshow

In [ ]:
imshow(frame)

In [ ]:
row, col, time, channel, plane

In [ ]:
filtered_df = df[(df['TimepointID'] == time)
                   &(df['PlaneID'] == plane)
                   &(df['ChannelID'] == channel)
                   &(df['Row'] == row)
                   &(df['Col'] == col)
                    ]

# Check segment

In [ ]:
v = napari.Viewer()
v.add_image(images[(3, 4)],
            channel_axis=1,
            name=["macrophage", "mtb"],
            colormap=["green", "magenta"],
            )  
# v.add_image(images[(3, 4)])
v.add_labels(masks[(3, 4)])